# Task 1 - Data Collection

In [33]:
import json, requests, urllib
from pathlib import Path
from datetime import datetime
import pandas as pd
from sodapy import Socrata
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import time

Connection parameters made up of a token which I signed up for at Tyler Data & Insights - https://evergreen.data.socrata.com/.  plus the endpoint and prefix which are a free API on the CDC website :  https://data.cdc.gov/NCHS/Provisional-COVID-19-Deaths-by-Sex-and-Age/9bhg-hcku

In [34]:
app_token =  '26qtuWJsZKYcQJCmkfrKQd1Pc'
api_endpoint = '9bhg-hcku'
api_prefix = 'data.cdc.gov'
connection = Socrata(api_prefix,app_token)

In [35]:
dir_raw = Path('raw_contents')
dir_raw.mkdir(parents =True,exist_ok=True)

The condition below is SOSQL  passed to a variable which is then passed as an argument in the connection function.  This allows us to only select only those rows of interest.  If we do not pass the condition we runt he risk of downloading the entire datset of 127 000 rows.  The dataset at CDC is grouped by Total, By year and by Mont.  So that is redundant and here I decided to only analyse by year and as such created a condition which only selects all those rows which are grouped by year.

In [36]:
condition = 'year IS NOT NULL AND month IS NULL'

We create a function that fetches the datset of interest. We ensure to use connection timeout and the sleep function so as to prevent our ip address from being flaggd for abuse.  Using some examples from the assignment specification document I came up with a way to create file names and then dump all files to the relevant folder.

Notice the offset argument which works with the limit argument to  allow us to page our rows at intervals until allows of interest have been saved.

In [37]:
def fetch_records(setName,paging):
    offset_count = 2700
    i = 0   
    data_set = [0]
    while len(data_set) > 0  :   
       
        connection.timeout = 60
        data_set = connection.get(api_endpoint,where=condition,order="year",offset=offset_count,limit=paging)
        if len(data_set) == 0:
             print('dataset download completed')
             break
        print('length of dataset ',len(data_set))
        #date_suffix = datetime.now().strftime('%Y%m%d-%H00')
        fileName = '%s.json' % (setName+str(i))
        filePath = dir_raw/fileName
        #print(filePath)
        print('Please Wait...........Generating file ',filePath)
        print('================================================')
        fileOut = open(filePath,'w')
        json.dump(data_set,fileOut)
        fileOut.close()
        time.sleep(30)
        offset_count = offset_count + paging
        i = i + 1
        print('offset value ',offset_count)      
 
      
                                 
      

We set the name of the dataset and together with the paging value wepass to the fetch_records function.

In [38]:
setName = 'covidSet-'
paging  = 2500
fetch_records(setName,paging)

length of dataset  2500
Please Wait...........Generating file  raw_contents\covidSet-0.json
offset value  5200
length of dataset  2500
Please Wait...........Generating file  raw_contents\covidSet-1.json
offset value  7700
length of dataset  2500
Please Wait...........Generating file  raw_contents\covidSet-2.json
offset value  10200
length of dataset  816
Please Wait...........Generating file  raw_contents\covidSet-3.json
offset value  12700
dataset download completed


Here we test our files by passing one file at a time.  It allows us to choose any file we wish to test. we then return as a dataframe and print it out to verify that what we needed is what we downloaded.

In [39]:
def test_data_sets(fname):
    covidset = json.load(open(dir_raw/fname))
    df = pd.DataFrame(covidset)
    return df

In [40]:
fname = 'covidSet-3.json'
test_data_sets(fname)

,data_as_of,start_date,end_date,group,year,state,sex,age_group,covid_19_deaths,total_deaths,pneumonia_deaths,pneumonia_and_covid_19_deaths,influenza_deaths,pneumonia_influenza_or_covid,footnote
0,2023-03-15T00:00:00.000,2023-01-01T00:00:00.000,2023-03-11T00:00:00.000,By Year,2023,Indiana,All Sexes,All Ages,435,11205,802,162,70,1142,NaN
1,2023-03-15T00:00:00.000,2023-01-01T00:00:00.000,2023-03-11T00:00:00.000,By Year,2023,Minnesota,Female,85 years and over,141,1849,106,28,10,227,NaN
2,2023-03-15T00:00:00.000,2023-01-01T00:00:00.000,2023-03-11T00:00:00.000,By Year,2023,Illinois,All Sexes,All Ages,729,20616,1294,271,67,1815,NaN
3,2023-03-15T00:00:00.000,2023-01-01T00:00:00.000,2023-03-11T00:00:00.000,By Year,2023,Mississippi,Female,85 years and over,63,887,71,16,NaN,121,One or more data cells have counts between 1-9...
4,2023-03-15T00:00:00.000,2023-01-01T00:00:00.000,2023-03-11T00:00:00.000,By Year,2023,Missouri,Female,85 years and over,92,2004,121,18,12,205,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,2023-03-15T00:00:00.000,2023-01-01T00:00:00.000,2023-03-11T00:00:00.000,By Year,2023,Michigan,Female,55-64 years,35,886,74,15,NaN,103,One or more data cells have counts between 1-9...
812,2023-03-15T00:00:00.000,2023-01-01T00:00:00.000,2023-03-11T00:00:00.000,By Year,2023,Michigan,Female,65-74 years,65,1649,117,17,16,180,NaN
813,2023-03-15T00:00:00.000,2023-01-01T00:00:00.000,2023-03-11T00:00:00.000,By Year,2023,Michigan,Female,75-84 years,113,2390,161,32,21,263,NaN
814,2023-03-15T00:00:00.000,2023-01-01T00:00:00.000,2023-03-11T00:00:00.000,By Year,2023,Texas,All Sexes,All Ages,1685,38270,2981,818,145,3973,NaN
